In [ ]:
목표6 : 소스 정리 및 learningrate 등의 값 조절 등을 하면서 실험

목표5 : batch 처리
    
목표4 : 파일/라벨의 수를 실제처럼 확대

목표3 : pooling, dropout을 넣어보자

목표2 : crossentroy를 해서 loss를 줄이는 처리

목표1 : 이미지를 하나 읽어 들이고 conv layer 2번 + fc layer 1번 처리

In [1]:
import tensorflow as tf
import os
from PIL import Image

In [2]:
image_dir = 'c:/tmp/Temp_data_Set/Test_Dataset_png/'
image_list = os.listdir(image_dir)
image_list.sort()    # sort()는 return이 없음

for i in range(len(image_list)):
    image_list[i] = image_dir + image_list[i]
    if i%1000 == 0:
        print(image_list[i])

label_dir = 'c:/tmp/Temp_data_Set/Test_Dataset_csv/label.csv'
label_list = [label_dir]
print(label_list)

imagename_queue = tf.train.string_input_producer(image_list)
labelname_queue = tf.train.string_input_producer(label_list)

c:/tmp/Temp_data_Set/Test_Dataset_png/Face00001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face01001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face02001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face03001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face04001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face05001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face06001.png
c:/tmp/Temp_data_Set/Test_Dataset_png/Face07001.png
['c:/tmp/Temp_data_Set/Test_Dataset_csv/label.csv']


In [3]:
image_width = 49
image_height = 61

In [4]:
image_reader = tf.WholeFileReader()
label_reader = tf.TextLineReader()
image_key, image_value = image_reader.read(imagename_queue)
label_key, label_value = label_reader.read(labelname_queue)

In [5]:
image_decoded = tf.cast(tf.image.decode_png(image_value), tf.float32)
label_decoded = tf.cast(tf.decode_csv(label_value, record_defaults=[[0]]), tf.float32)
#print(image)

label = tf.reshape(label_decoded,[1])
image = tf.reshape(image_decoded,[image_height, image_width, 1])

x, y_ = tf.train.shuffle_batch(tensors=[image, label], batch_size=32, num_threads=4, capacity=5000, min_after_dequeue=100)

In [6]:
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32]))
b_conv1 = tf.Variable(tf.zeros([32]))

W_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64]))
b_conv2 = tf.Variable(tf.zeros([64]))

W_fc1 = tf.Variable(tf.truncated_normal([image_width*image_height*64, 10]))
b_fc1 = tf.Variable(tf.zeros([10]))

W_fc2 = tf.Variable(tf.truncated_normal([10, 1]))
b_fc2 = tf.Variable(tf.zeros([1]))

In [7]:
x_image = tf.reshape(x, [-1, image_width, image_height, 1])

keep_prob = tf.placeholder(tf.float32)

In [8]:
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding="SAME") + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding="SAME") + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")
h_pool2_flat = tf.reshape(h_pool2, [-1, image_width*image_height*64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [9]:
print(y_conv)
print(y_)

Tensor("add_3:0", shape=(32, 1), dtype=float32)
Tensor("shuffle_batch:1", shape=(32, 1), dtype=float32)


In [10]:
loss = tf.reduce_min(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y_conv))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [11]:
# prediction한 값과 label값을 비교 : True/False 리스트
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

# True/False 리스트를 실수형으로 casting한다.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
with tf.Session() as sess:
    print("started")
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    sess.run(tf.global_variables_initializer())

    for i in range(100):
#        print(sess.run(image_key))
#        print(sess.run(image_key))
        sess.run(train, {keep_prob: 0.7})
#        sess.run(train)
        
        #if train==True:
        _looss = sess.run(loss, {keep_prob: 0.7})
        _accuracy = sess.run(accuracy, {keep_prob: 0.7})
        #if release_mode==Train:
        #result = sess.run(pred, {keep_prob: 1.0})
        
        print("-----------------")
        print("loss: ", _looss)
        print("accuracy: ", _accuracy)
        
#    print(image)

#    Image.fromarray(image).show()

    print("training done")
    
    coord.request_stop()
    coord.join(thread)

started
-----------------
loss:  -4.9964e+08
accuracy:  1.0
-----------------
loss:  -3.3631e+08
accuracy:  1.0
-----------------
loss:  -6.09782e+08
accuracy:  1.0
-----------------
loss:  -4.47982e+08
accuracy:  1.0
-----------------
loss:  -3.95813e+08
accuracy:  1.0
-----------------
loss:  -4.67165e+08
accuracy:  1.0
-----------------
loss:  -5.31558e+08
accuracy:  1.0
-----------------
loss:  -4.06572e+08
accuracy:  1.0
-----------------
loss:  -5.50922e+08
accuracy:  1.0
-----------------
loss:  -4.8031e+08
accuracy:  1.0
-----------------
loss:  -3.17638e+08
accuracy:  1.0
-----------------
loss:  -6.93324e+08
accuracy:  1.0
-----------------
loss:  -6.08204e+08
accuracy:  1.0
-----------------
loss:  -5.95149e+08
accuracy:  1.0
-----------------
loss:  -5.50654e+08
accuracy:  1.0
-----------------
loss:  -7.95836e+08
accuracy:  1.0
-----------------
loss:  -7.59806e+08
accuracy:  1.0
-----------------
loss:  -7.88546e+08
accuracy:  1.0
-----------------
loss:  -5.28386e+08
acc